# Parameter Estimation using basico


**Description** <br>
Parameter estimation (parameter fitting) is a process of finding the best set of parameters for a model to fit the experimental data. Basico enables both local and global parameter estimation algorithms.

**Setup** <br>
To accomplish parameter estimation task, the following components are required:
* model *(ODE model in SBML format)*
* experimental data *(time series in a table format)*
* parameters to be estimated *(can be both kinetic parameters and initial concentrations)*
* algorithms for the parameter estimation task (see here: https://basico.readthedocs.io/en/latest/API/basico.html#basico.task_parameterestimation.run_parameter_estimation)
* available range (constraints) for the parameters

**Parameter estimation workflow from users perspective** <br
Choose the initial parameter values and ranges
Choose the algorithm
Run the parameter fitting
Visually inspect the plot of data vs fit
Review the table of fit statistics with parameters basico.get_fit_statistic(include_parameters=True)
Repeat 1 - 5


1. Load model and remove previous experiments (`basico.remove_experiments()`)
2. Add experimental data (`basico.add_experiment(name, data)`)
3. Add reaction parameters to be estimated together with their upper and lower bounds <br>
Create a list of dictionaries where each dictionary contains the name of the parameter, its lower and upper bounds. <br>
E.g.
```
fit_items = [
            {'name': '(R1).k1', 'lower': 0.001, 'upper': 2},
            {'name': '(R2).k1', 'lower': 0.001, 'upper': 2},
            {'name': '(R3).k1', 'lower': 0.001, 'upper': 2},
            {'name': '(R4).k1', 'lower': 0.001, 'upper': 2},
        ]	
```
4. Set parameters to be estimated (`basico.set_fit_parameters(fit_items)`) <br>
4.1 (optional) Constrains for the concentrations. Solutions with concentrations outside a certain range are rejected. <br>
5. Run parameter estimation (`basico.run_parameter_estimation(method='Levenberg - Marquardt', update_model=True)`). If the  `update_model` parameter is set to `True`, the model will be updated with the estimated parameters, thus it is possible to restart the estimation process from the same point in the parameter space, but with different algorithm.
6. Evaluate the results <br>
6.1 Visually inspect the plot of data vs fit (run model simulation with the estimated parameters)
6.2 Inspect fit statistics, parameter values and the objective function value. <br>
`basico.get_fit_statistic(include_parameters=True)`
7. Repeat points 4 - 6 until the desired outcome is reached.

**Visualisation and ouput** <br>
The progress of the parameter estimation can be tracked visually by observing the objective value (y -axis) over number of iterations (x-axis) of the estimation algorithm, ideally, as a curve. Also output of the estimated values in a table where *before* and *after* values can be compared would be beneficial.

**Testing** <br>
Use the example below ('brusselator') to see if the parameter estimation is running correctly. Ideally, the estimation should end with exactly the original parameters of this example. The estimation process can be time and resource-consuming. For testing it is important to use local methods, such as , e.g. `Levenberg - Marquardt` or set **the seed** for global methods, since global methods use stochastic algorithms to set initial starting point for the estimation.

**Note** <br>
Since parameter estimation task is usually an interative process, the user should be able to stop the estimation process at any time, evaluate the results, and resume it with another algorithm or with the same algorithm.


# Parameter estimation task

In [32]:
import os
import pandas as pd
import basico
import matplotlib.pyplot as plt
import numpy as np

## 0 Generate mock data
For demonstration and testing purpose.

In [33]:
# load model and remove experimental data
basico.load_example('brusselator')
# remove previous experiments to avoid contamination
basico.remove_experiments()

In [34]:
# create noisy data from the model, that we will take for a parameter estimation
basico.add_parameter('obs_x', type='assignment', expression='[X] + UNIFORM(0,1) - 0.5')
basico.add_parameter('obs_y', type='assignment', expression='[Y] + UNIFORM(0,1) - 0.5');

# run a time course to generate time-series
result = basico.run_time_course(start_time=0, use_number=True)

#  generated time-series
# the "mock" experimental data is under Values[obs_x] and Values[obs_y]
result.head()

,X,Y,Values[obs_x],Values[obs_y]
Time,,,,
0.0,2.999996,2.999996,2.944745,2.658725
0.5,3.408155,0.817484,3.312834,0.860739
1.0,1.896454,1.276790,2.329036,1.289104
1.5,0.876253,1.872929,1.127705,1.871687
2.0,0.345934,2.368188,0.194957,2.430537


In [35]:
# clean up the "experimental data"
data = result.drop(columns=['X', 'Y'])
data.rename(columns = {'Values[obs_x]':'[X]', 'Values[obs_y]':'[Y]'}, inplace=True)
data = data.reset_index()

# clean up parameter values
basico.remove_parameter('obs_x')
basico.remove_parameter('obs_y')

# sow cleaned version
data.head()

,Time,[X],[Y]
0,0.0,2.944745,2.658725
1,0.5,3.312834,0.860739
2,1.0,2.329036,1.289104
3,1.5,1.127705,1.871687
4,2.0,0.194957,2.430537


#### Perturb model parameters

In [36]:
# initial parameters
basico.get_reaction_parameters()

,value,reaction,type,mapped_to
name,,,,
(R1).k1,1.0,R1,local,
(R2).k1,1.0,R2,local,
(R3).k1,1.0,R3,local,
(R4).k1,1.0,R4,local,


In [37]:
# change parameters so we have a starting point for the estimation task
basico.set_reaction_parameters(['(R1).k1', '(R2).k1', '(R3).k1', '(R4).k1'], value=0.5)

# view simulation of the edited values
#basico.run_time_course(start_time=0).plot();

In [38]:
# review if the setting was successful
basico.get_reaction_parameters()

,value,reaction,type,mapped_to
name,,,,
(R1).k1,0.5,R1,local,
(R2).k1,0.5,R2,local,
(R3).k1,0.5,R3,local,
(R4).k1,0.5,R4,local,


## 1. Add experimental data to a model

In [ ]:
# add experimental data to the estimation task
basico.add_experiment('exp1', data)

'c:\\Users\\Wehling\\code\\feat-estimation\\AIAgents4Pharma\\docs\\notebooks\\talk2biomodels\\exp1.txt'

In [ ]:
#check if the mapping was correct
basico.get_experiment_mapping('exp1')

,type,mapping,cn,column_name
column,,,,
0,time,,,Time
1,dependent,[X],"CN=Root,Model=The Brusselator,Vector=Compartme...",[X]
2,dependent,[Y],"CN=Root,Model=The Brusselator,Vector=Compartme...",[Y]


In [ ]:
# Review how experimental data maps to the current simulation
basico.plot_per_experiment();

NameError: name 'basico' is not defined

##  2. Choose parameters to be estimated and the initial parameter values and ranges

In [39]:
# adding reaction parameters to the experiment
fit_items = [
            {'name': '(R1).k1', 'lower': 0.001, 'upper': 2},
            {'name': '(R2).k1', 'lower': 0.001, 'upper': 2},
            {'name': '(R3).k1', 'lower': 0.001, 'upper': 2},
            {'name': '(R4).k1', 'lower': 0.001, 'upper': 2},
        ]


In [40]:
basico.set_fit_parameters(fit_items)
basico.get_fit_parameters()

,lower,upper,start,affected,cn
name,,,,,
(R1).k1,0.001,2,0.5,[],"CN=Root,Model=The Brusselator,Vector=Reactions..."
(R2).k1,0.001,2,0.5,[],"CN=Root,Model=The Brusselator,Vector=Reactions..."
(R3).k1,0.001,2,0.5,[],"CN=Root,Model=The Brusselator,Vector=Reactions..."
(R4).k1,0.001,2,0.5,[],"CN=Root,Model=The Brusselator,Vector=Reactions..."


In [41]:
# here's how data looks like
data.head()

,Time,[X],[Y]
0,0.0,2.944745,2.658725
1,0.5,3.312834,0.860739
2,1.0,2.329036,1.289104
3,1.5,1.127705,1.871687
4,2.0,0.194957,2.430537


### 2.1 Set constraints for the concentrations


In [42]:
#setting constraints for the fitting task
# solutions with concentrations outside a certain range are rejected
basico.set_fit_constraints([
    {'name': 'Y', 'lower': 0, 'upper': 10}
])

## 3. Run parameter estimation task

Choose algorithms form :
 
 * Current Solution
 
 * Current Solution Statistics

Global Methods:

* Random Search

* Simulated Annealing

* Differential Evolution

* Scatter Search

* Genetic Algorithm

* Evolutionary Programming

* Genetic Algorithm SR

* Evolution Strategy (SRES)

* Particle Swarm

Local Methods:

*    Levenberg - Marquardt,

*    Hooke & Jeeves,

*    Nelder - Mead,

*    Steepest Descent,

*    NL2SOL,

*    Praxis,

*    Truncated Newton,

In [46]:
basico.run_parameter_estimation(method='Levenberg - Marquardt', update_model=True)

,lower,upper,sol,affected
name,,,,
(R1).k1,0.001,2,0.644231,[]
(R2).k1,0.001,2,0.567104,[]
(R3).k1,0.001,2,0.479191,[]
(R4).k1,0.001,2,0.890338,[]


In [47]:
basico.run_parameter_estimation(method='Evolution Strategy (SRES)', update_model=True)

,lower,upper,sol,affected
name,,,,
(R1).k1,0.001,2,1.092313,[]
(R2).k1,0.001,2,0.718337,[]
(R3).k1,0.001,2,0.836153,[]
(R4).k1,0.001,2,1.022308,[]


## 4. Evaluate results


In [48]:
#Review fit statistics
basico.get_fit_statistic(include_parameters=False)

{'obj': 18.17203498427607,
 'rms': 0.2126124811130919,
 'sd': 0.2136782145130326,
 'f_evals': 23908,
 'failed_evals_exception': 0,
 'failed_evals_nan': 0,
 'constraint_evals': 23901,
 'failed_constraint_evals': 1492,
 'cpu_time': 8.609375,
 'data_points': 402,
 'valid_data_points': 402,
 'evals_per_sec': 0.0003601043583737661}

### Loading data from a file


In [49]:
# load data from file
exp_file = "exp1.txt"
exp_data = pd.read_csv(exp_file, sep="\t") # Ensure correct separator
